In [2]:
# Importing necessary Libraries

In [3]:
from IPython.display import clear_output
!pip install imutils
clear_output()

In [1]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import scipy

import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *

# import pydot

from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K

from tqdm import tqdm, tqdm_notebook
from colorama import Fore
import json
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
%config Completer.use_jedi = False
import time
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

init_notebook_mode(connected=True)
RANDOM_SEED = 123

print("All modules have been imported")

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (c:\Users\ishra\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\preprocessing\image\__init__.py)

In [ ]:
!apt-get install tree
clear_output()
# create new folders
!mkdir TRAIN TEST VAL TRAIN/YES TRAIN/NO TEST/YES TEST/NO VAL/YES VAL/NO
!tree -d

In [ ]:
IMG_PATH = "../input/brain-tumor-detection-mri/Brain_Tumor_Detection"

# split the data by train/val/test
ignored = {"pred"}
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if CLASS not in ignored:
        if not CLASS.startswith("."):
            IMG_NUM = len(os.listdir(IMG_PATH + "/" + CLASS))
            for n, FILE_NAME in enumerate(os.listdir(IMG_PATH + "/" + CLASS)):
                img = IMG_PATH + "/" + CLASS + "/" + FILE_NAME
                if n < 300:
                    shutil.copy(img, "TEST/" + CLASS.upper() + "/" + FILE_NAME)
                elif n < 0.8 * IMG_NUM:
                    shutil.copy(img, "TRAIN/" + CLASS.upper() + "/" + FILE_NAME)
                else:
                    shutil.copy(img, "VAL/" + CLASS.upper() + "/" + FILE_NAME)

In [ ]:
def load_data(dir_path, img_size=(100, 100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith("."):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith("."):
                    img = cv2.imread(dir_path + path + "/" + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f"{len(X)} images loaded from {dir_path} directory.")
    return X, y, labels


def plot_confusion_matrix(
    cm, classes, normalize=False, title="Confusion matrix", cmap=plt.cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.0
    cm = np.round(cm, 2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

In [ ]:
TRAIN_DIR = "TRAIN/"
TEST_DIR = "TEST/"
VAL_DIR = "VAL/"
IMG_SIZE = (224, 224)

In [ ]:
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

In [ ]:
# Plotting the number of samples in Training, Validation and Test sets

In [ ]:
y = dict()
y[0] = []
y[1] = []
for set_name in (y_train, y_val, y_test):
    y[0].append(np.sum(set_name == 0))
    y[1].append(np.sum(set_name == 1))

trace0 = go.Bar(
    x=["Train Set", "Validation Set", "Test Set"],
    y=y[0],
    name="No",
    marker=dict(color="#33cc33"),
    opacity=0.7,
)
trace1 = go.Bar(
    x=["Train Set", "Validation Set", "Test Set"],
    y=y[1],
    name="Yes",
    marker=dict(color="#ff3300"),
    opacity=0.7,
)
data = [trace0, trace1]
layout = go.Layout(
    title="Count of classes in each set",
    xaxis={"title": "Set"},
    yaxis={"title": "Count"},
)
fig = go.Figure(data, layout)
iplot(fig)

In [ ]:
# Let's Visualize the images we are working with

In [ ]:
def plot_samples(X, y, labels_dict, n=50):
    """
    Creates a gridplot for desired number of images (n) from the specified set
    """
    for index in range(len(labels_dict)):
        imgs = X[np.argwhere(y == index)][:n]
        j = 10
        i = int(n / j)

        plt.figure(figsize=(15, 6))
        c = 1
        for img in imgs:
            plt.subplot(i, j, c)
            plt.imshow(img[0])

            plt.xticks([])
            plt.yticks([])
            c += 1
        plt.suptitle("Tumor: {}".format(labels_dict[index]))
        plt.show()

In [ ]:
plot_samples(X_train, y_train, labels, 30)

In [ ]:
# Cropping the images

The images we have are of different sizes. But our model accepts images of size (224*224*3) as input. To achienve this ew have to resize the images. Blindly resizing the images can lead to extreme distortions in the images. Hence, We will first crop thie images and then resize them. This will minimize the issue of distortions.

This cropping is done by finding contours in the images using the OpenCV Library

In [ ]:
def crop_imgs(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(
            thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)
        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[
            extTop[1] - ADD_PIXELS : extBot[1] + ADD_PIXELS,
            extLeft[0] - ADD_PIXELS : extRight[0] + ADD_PIXELS,
        ].copy()
        set_new.append(new_img)

    return np.array(set_new)

In [ ]:
import imutils

img = cv2.imread("./VAL/NO/no852.jpg")
img = cv2.resize(img, dsize=IMG_SIZE, interpolation=cv2.INTER_CUBIC)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# threshold the image, then perform a series of erosions +
# dilations to remove any small regions of noise
thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# find contours in thresholded image, then grab the largest one
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)

# find the extreme points
extLeft = tuple(c[c[:, :, 0].argmin()][0])
extRight = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])

# add contour on the image
img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

# add extreme points
img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

# crop
ADD_PIXELS = 0
new_img = img[
    extTop[1] - ADD_PIXELS : extBot[1] + ADD_PIXELS,
    extLeft[0] - ADD_PIXELS : extRight[0] + ADD_PIXELS,
].copy()

In [ ]:
# Let's visualize how the cropping works

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(141)
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.title("Step 1. Get the original image")
plt.subplot(142)
plt.imshow(img_cnt)
plt.xticks([])
plt.yticks([])
plt.title("Step 2. Find the biggest contour")
plt.subplot(143)
plt.imshow(img_pnt)
plt.xticks([])
plt.yticks([])
plt.title("Step 3. Find the extreme points")
plt.subplot(144)
plt.imshow(new_img)
plt.xticks([])
plt.yticks([])
plt.title("Step 4. Crop the image")
plt.show()

In [ ]:
X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)

In [ ]:
# Let's visualize the images after being cropped

In [ ]:
plot_samples(X_train_crop, y_train, labels, 30)

In [ ]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for img, imclass in zip(x_set, y_set):
        if imclass == 0:
            cv2.imwrite(folder_name + "NO/" + str(i) + ".jpg", img)
        else:
            cv2.imwrite(folder_name + "YES/" + str(i) + ".jpg", img)
        i += 1

In [ ]:
# saving new images to the folder
!mkdir TRAIN_CROP TEST_CROP VAL_CROP TRAIN_CROP/YES TRAIN_CROP/NO TEST_CROP/YES TEST_CROP/NO VAL_CROP/YES VAL_CROP/NO

save_new_images(X_train_crop, y_train, folder_name='TRAIN_CROP/')
save_new_images(X_val_crop, y_val, folder_name='VAL_CROP/')
save_new_images(X_test_crop, y_test, folder_name='TEST_CROP/')

In [ ]:
# Resizing the images

In [ ]:
def preprocess_imgs(set_name, img_size):
    set_new = []
    for img in set_name:
        img = cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_CUBIC)
        set_new.append(preprocess_input(img))
    return np.array(set_new)

In [ ]:
X_train_prep = preprocess_imgs(set_name=X_train_crop, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test_crop, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val_crop, img_size=IMG_SIZE)

In [ ]:
plot_samples(X_train_prep, y_train, labels, 30)

In [ ]:
# IMAGE AUGMENTATION

In [ ]:
demo_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1.0 / 255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
)

os.mkdir("preview")
x = X_train_crop[0]
x = x.reshape((1,) + x.shape)

i = 0
for batch in demo_datagen.flow(
    x, batch_size=1, save_to_dir="preview", save_prefix="aug_img", save_format="jpg"
):
    i += 1
    if i > 20:
        break

plt.imshow(X_train_crop[0])
plt.xticks([])
plt.yticks([])
plt.title("Original Image")
plt.show()

plt.figure(figsize=(15, 6))
i = 1
for img in os.listdir("preview/"):
    img = cv2.cv2.imread("preview/" + img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(3, 7, i)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    i += 1
    if i > 3 * 7:
        break
plt.suptitle("Augemented Images")
plt.show()

In [ ]:
!rm -rf preview/

In [ ]:
TRAIN_DIR = "TRAIN_CROP/"
VAL_DIR = "VAL_CROP/"
RANDOM_SEED = 42
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode="rgb",
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode="binary",
    seed=RANDOM_SEED,
)
validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode="rgb",
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode="binary",
    seed=RANDOM_SEED,
)

In [ ]:
# Creating the model

In [ ]:
base_Neural_Net = VGG19(
    input_shape=(224, 224, 3), weights="imagenet", include_top=False
)
model = Sequential()
model.add(base_Neural_Net)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

for layer in base_Neural_Net.layers:
    layer.trainable = False


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"])

model.summary()

In [ ]:
# TRAINING !!!

In [ ]:
EPOCHS = 30
es = EarlyStopping(monitor="val_acc", mode="max", patience=6)
history = model.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=25,
    callbacks=[es],
)

In [ ]:
# Let's have a look at the performance of our model

In [ ]:
# Validating with the training set

In [ ]:
predictions = model.predict(X_train_prep)
predictions = [1 if x > 0.5 else 0 for x in predictions]

accuracy = accuracy_score(y_train, predictions)
print("Train Accuracy = %.2f" % accuracy)

confusion_mtx = confusion_matrix(y_train, predictions)
cm = plot_confusion_matrix(confusion_mtx, classes=list(labels.items()), normalize=False)

In [ ]:
# Validating with the Validation set

In [ ]:
predictions = model.predict(X_val_prep)
predictions = [1 if x > 0.5 else 0 for x in predictions]

accuracy = accuracy_score(y_val, predictions)
print("Val Accuracy = %.2f" % accuracy)

confusion_mtx = confusion_matrix(y_val, predictions)
cm = plot_confusion_matrix(confusion_mtx, classes=list(labels.items()), normalize=False)

In [ ]:
# Validating with the Test set

In [ ]:
# validate on test set
predictions = model.predict(X_test_prep)
predictions = [1 if x > 0.5 else 0 for x in predictions]

accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy = %.2f" % accuracy)

confusion_mtx = confusion_matrix(y_test, predictions)
cm = plot_confusion_matrix(confusion_mtx, classes=list(labels.items()), normalize=False)

In [ ]:
prob_pred = model.predict_proba(X_test_prep)

In [ ]:
# Let's see the other performance metrics on the test set

In [ ]:
from sklearn import metrics

print("Accuracy score is :", metrics.accuracy_score(y_test, predictions))
print(
    "Precision score is :",
    metrics.precision_score(y_test, predictions, average="weighted"),
)
print(
    "Recall score is :", metrics.recall_score(y_test, predictions, average="weighted")
)
print("F1 Score is :", metrics.f1_score(y_test, predictions, average="weighted"))
print(
    "ROC AUC Score is :",
    metrics.roc_auc_score(y_test, prob_pred, multi_class="ovo", average="weighted"),
)
print("Cohen Kappa Score:", metrics.cohen_kappa_score(y_test, predictions))
print(
    "\t\tClassification Report:\n", metrics.classification_report(y_test, predictions)
)